In [9]:
import chromadb
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    Settings,
    ChatPromptTemplate,
    PromptTemplate,
    StorageContext,
    get_response_synthesizer,
)
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.llms.llama_cpp.llama_utils import (
    messages_to_prompt,
    completion_to_prompt,
)

from llama_index.llms.openai import OpenAI

import sys
import os
import openai

sys.path.append("..")
from src.constants import qa_prompt_tmpl_str, refine_prompt_tmpl_str, context_tmpl_str
from dotenv import load_dotenv

load_dotenv()

True

In [10]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [11]:
embed_model = HuggingFaceEmbedding("intfloat/multilingual-e5-base", device="cuda")

In [12]:
# llm = LlamaCPP(
#     model_path="../src/weights/saiga3_v6_model-q8_0.gguf",
#     context_window=4096,
#     max_new_tokens=512,
#     model_kwargs={"n_gpu_layers": -1},
#     temperature=0,
#     messages_to_prompt=messages_to_prompt,
#     completion_to_prompt=completion_to_prompt,
#     verbose=True,
# )

In [24]:
llm = OpenAI(
    model="gpt-4o",
)

In [25]:
Settings.llm = llm
Settings.chunk_size = 512
Settings.embed_model = embed_model
Settings.chunk_overlap = 256

In [26]:
# qa_prompt_tmpl = PromptTemplate(qa_prompt_tmpl_str)
# refine_prompt_tmpl = PromptTemplate(refine_prompt_tmpl_str)

In [27]:
documents = SimpleDirectoryReader(input_dir="../data/docs/").load_data()
db = chromadb.PersistentClient(path="../chroma_db_law_test")
chroma_collection = db.get_or_create_collection("quickstart")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, embed_model=embed_model
)

In [28]:
chat_engine = index.as_chat_engine(
    chat_mode="context", context_template=context_tmpl_str
)

In [31]:
response = chat_engine.stream_chat("О чем этот документ?")

In [32]:
for token in response.response_gen:
    print(token, end="")

Документ представляет собой письмо Федеральной службы по надзору в сфере природопользования, в котором говорится о возможности отзыва разрешения в случае нарушения обязательных требований в течение 6 месяцев и более, а также в случае снятия объекта негативного воздействия на окружающую среду с государственного учета.